# Bolsonaro vs Lula

Este estudo é uma análise de sentimento de tweets coletados contentado a palavra Bolsonaro, Lula e ambos. O intuito é aproveitar este momento de início da campanha eleitoral e realizar um tipo de termômetro do que é falado a respeito dos candidatos no Twitter.

Não é de interesse deste experimento fazer campanha para algum candidato ou contra este.

## IMPORTS

In [1]:
import pandas as pd
import re
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from string import punctuation
import joblib
from nltk import tokenize
import nltk
from string import punctuation
from unidecode import unidecode
import unidecode
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

## Carregando o modelo

In [4]:
modelo = joblib.load('./model/nb_predictor.joblib')
with open('./model/tfidf.pkl', 'rb') as file:
    feature_vocabulary = pickle.load(file)

In [5]:
import pickle

# Load the feature vocabulary from feature.pkl
with open('./model/tfidf.pkl', 'rb') as file:
    feature_vocabulary = pickle.load(file)

# Now, feature_vocabulary contains the vocabulary from feature.pkl
tfidf = TfidfVectorizer()
vidf = tfidf.fit_transform(feature_vocabulary)

## FUNÇÕES

### Limpando chars 

In [13]:

def limpando_chars(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # Removendo menções
    text = re.sub(r'#\w+', '', text) # Removendo #hashtags
    text = re.sub(r'#', '', text)
    text = re.sub(r'\r', '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'_', '', text)
    text = re.sub(r'RT[\s]+', '', text)
    text = re.sub(r'https?:\/\/\S+', '', text)
    text = re.sub(r'\n', '', text)
    text = text.lower()
    return text


### Porcessar texto

In [6]:
def processar_texto(texto):
    stemmer = nltk.RSLPStemmer()
    stopwords = nltk.corpus.stopwords.words('portuguese')
    texto = re.sub(r'@[A-Za-z0-9]+', '', texto) # Removendo menções
    texto = re.sub(r'#\w+', '', texto) # Removendo #hashtags
    texto = re.sub(r'#', '', texto)
    texto = re.sub(r'_', '', texto)
    texto = re.sub(r'RT[\s]+', '', texto)
    texto = re.sub(r'https?:\/\/\S+', '', texto)
    texto = re.sub(r'\n', '', texto)
    pontuacao = []
    for ponto in punctuation:
        pontuacao.append(ponto)
    # Remove caracteres especiais
    texto = re.sub(r'[^\w\s]', '', texto)
    # Converte para minúsculas
    texto = texto.lower()
    texto = [stemmer.stem(palavra) for palavra in texto.split() if palavra not in stopwords]
    # Une acentos
    texto = [unidecode.unidecode(palavra) for palavra in texto]
    return ' '.join(texto)

### Predict it

In [8]:
def predict_it(frase):
    # Aplica a função de pré-processamento
    texto_processado = processar_texto(frase)
    # Transforma o texto em um vetor TF-IDF
    vetor_tfidf = tfidf.transform([texto_processado])
    # Prevê a classificação da frase
    previsao = modelo.predict(vetor_tfidf)
    probabilidades = modelo.predict_proba(vetor_tfidf).astype('float')[0]
    pos = probabilidades[1]
    neg = probabilidades[0]
    if pos < 0.51 and neg < 0.51:
        return 'Neutro'
    else:
        return previsao.astype('str')[0]

## Import tweets Bolsonaro

In [9]:
# Crie uma lista com os nomes dos arquivos CSV.
arquivos = [f"./SNScraper_Data/Bolsonaro-{i}.csv" for i in range(1, 32)]

# Crie um dataframe vazio.
dfBolsonaro = pd.DataFrame()

# Concatena os arquivos CSV em um único dataframe.
dfBolsonaro = pd.concat([pd.read_csv(arquivo) for arquivo in arquivos])

dfBolsonaro['Candidato'] = 'Bolsonaro'


## Import tweets Lula

In [10]:
# Crie uma lista com os nomes dos arquivos CSV.
arquivos = [f"./SNScraper_Data/Lula-{i}.csv" for i in range(1, 32)]

# Crie um dataframe vazio.
dfLula = pd.DataFrame(index=None)

# Concatena os arquivos CSV em um único dataframe.
dfLula = pd.concat([pd.read_csv(arquivo) for arquivo in arquivos])

dfLula['Candidato'] = 'Lula'


## Concatenando DF's

In [11]:

dfs = [dfBolsonaro, dfLula]
df = pd.concat(dfs, ignore_index=True)
df = df[['Data', 'User', 'Texto', 'Candidato']]

In [14]:
df['pre-processed'] = df['Texto'].apply(limpando_chars)
dfBolsonaro['pre-processed'] = dfBolsonaro['Texto'].apply(limpando_chars)
dfLula['pre-processed'] = dfLula['Texto'].apply(limpando_chars)

## REALIZANDO PREDIÇÃO

In [15]:
df['Sentimento'] = df['Texto'].apply(predict_it)

In [18]:
df.head()

,Data,User,Texto,Candidato,pre-processed,Sentimento
0,2022-10-01 23:59:59+00:00,grandalsavino,Aqui no Paranoá vou assim...\r\n@DenianCouto @...,Bolsonaro,aqui no paranoá vou assim...,Negativo
1,2022-10-01 23:59:59+00:00,HellendaRochaV1,@denisoon_ Meus primos que nunca trabalharam n...,Bolsonaro,meus primos que nunca trabalharam na vida são...,Positivo
2,2022-10-01 23:59:58+00:00,antunes_shirlei,@Doutorapetista @lovato_po @jairbolsonaro @man...,Bolsonaro,po q horror chamar uma pessoa de câncer. el...,Negativo
3,2022-10-01 23:59:58+00:00,EstranhoDemais1,"@Marisete1982 @Fa1ryNight A luta é constante, ...",Bolsonaro,"a luta é constante, mas venceremos, fé em de...",Positivo
4,2022-10-01 23:59:58+00:00,riosvida6,Porque votar em Jair Bolsonaro neste domingo h...,Bolsonaro,porque votar em jair bolsonaro neste domingo,Negativo
